In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [12]:
gau_rages_page = requests.get("https://finalfantasy.wikia.com/wiki/Rage_(Final_Fantasy_VI_command)")
soup = BeautifulSoup(gau_rages_page.content, "html.parser")

In [19]:
list_of_rages_header = soup.find("span", id="List_of_Rages")
rage_table = list_of_rages_header.findNext("table")

In [140]:
all_table_th = rage_table.findAll("th")
base_link = "https://finalfantasy.wikia.com"
enemy_abilities_link = "https://finalfantasy.wikia.com/wiki/List_of_Final_Fantasy_VI_enemy_abilities"

abilities = requests.get(enemy_abilities_link)
abilities_soup = BeautifulSoup(abilities.content, "html.parser")

misspells_dict = {}
misspells_dict["Incisors"] = "Incisor"
misspells_dict["Landslide"] = "Land_Slide"

visited_id = set()

output_file = "spell_power.csv"
headers = ["Ability Name", "Magic Power", "Description"]
# with open(output_file, "w") as file:
#     for header in headers:
#         file.write(header + ",")
#     file.write("\n")

df = pd.DataFrame(columns=headers)
    
for i in range(7, len(all_table_th), 3):
    cur_th = all_table_th[i]
    a_link = cur_th.find("a").get('href')
    ability_link = "".join((base_link, a_link))

    # For now, only check the enemy abilities link
    if (enemy_abilities_link in ability_link):
        id_1 = ability_link.split("#")[1]
        # If misspelled
        if id_1 in misspells_dict:
            id_1 = misspells_dict[id_1]

        next_row = []
        if id_1 not in visited_id:
            visited_id.add(id_1)

            span_found = abilities_soup.find("span", id=id_1)
            # If we cannot find the span, we need to manually add it to misspells_dict
            if span_found is None:
                print(ability_link)

            next_td = str(span_found.findNext("td"))
            # if the next td has the word "spell power" in it
            spell_power = 0
            physical_power = 0

            if "spell power" in next_td.lower():
                index_of_spell = next_td.index("Spell Power")
                substr_1 = next_td[index_of_spell:index_of_spell + 25]
                temp_power = []
                # Iterate through substr until you find a digit
                for char in substr_1:
                    if char.isdigit():
                        temp_power.append(char)
                temp_power = int("".join(temp_power))

                next_row.append(id_1)
                next_row.append(temp_power)
                next_row.append(next_td)
                df.loc[len(df.index)] = next_row
                
                s = ",".join((str(id_1), str(temp_power), next_td, "\n"))
#                     file.write(s)
        

In [142]:
df2 = df.sort_values("Magic Power", ascending=False)
df2.to_csv(output_file)
df2

,Ability Name,Magic Power,Description
2,Metal_Cutter,120,<td>Deals non-elemental magic damage to one or...
7,Gigavolt,110,<td>Deals Lightning-elemental damage to one or...
6,Magnitude_8,100,<td>Deals Earth-elemental damage to all target...
3,Lifeshaver,84,"<td>Deals <a href=""/wiki/Earth_(element)"" titl..."
9,Land_Slide,75,<td>Deals Earth-elemental damage to all target...
8,Flash_Rain,60,<td>Deals Ice/Water dual-Elemental damage to a...
10,Fireball,50,<td>Deals Fire-elemental damage to all targets...
1,Snowstorm,25,<td>Deals Ice-Elemental damage to all targets....
0,Megavolt,20,<td>Deals Lightning-elemental damage to one or...
4,Venomist,20,<td>Deals Poison-elemental damage and inflicts...
